# CCRESS Training School - MUNICH 2025
### STRATfinder : an automated detection algorithm of the mixed layer height
#### NoteBook 1/2 : Signal Corrections
This code explores corrections that should be applied to attenuated backscatter observations observed with automatic lidars and ceiloeters (ALC). 

ALC continuously probe the atmosphere, operating 24/7, with a temporal resolution of seconds and a vertical resolution ranging from 5 to 15 meters depending on the instrument. 

Here, three ALC models are considered that are commonly deployed at ACTRIS NF: Lufft/Ott Hydromet CHM15k, Vaisala CL51, and Vaisala CL61.

INput files need to be processed with raw2l1 (M.-A. Drouin, SIRTA) - https://github.com/ACTRIS-CCRES/raw2l1.git - before applying STRATFinder.

Once the raw signals are corrected, the STRATFinder algorithm (Kotthaus et al. 2020) - https://gitlab.in2p3.fr/ipsl/sirta/mld/stratfinder/stratfinder.git - can be applied to detect and track the mixed layer height (MLH).

#### M. Van Hove, IPSL & S. Kotthaus, Ecole Polytechnique 
#### Contact : mvanhove@ipsl.fr

In [ ]:
#1
### ------- Import libraries ------- 

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import importlib

from datetime import date

from functions import from_df_to_pivot, plot_overlap_corrections, plot_MLH

In [ ]:
#2
### in case you edit the functions you need to reload them : 
import functions
importlib.reload(functions)
from functions import from_df_to_pivot, plot_overlap_corrections, plot_MLH
#import functions

In [ ]:
#3
## -------- Path to data ----------- 

path_data = "/data/ACTRIS-CCRES/abl_heights"


#### Load data
#### L1 Format : E-Profile, processed with raw2l1 


In [ ]:
#4
### =========== CHM15k ============
# The CHM15k provide attenuated backscatter from around 100m to 15km => it has a near-range blind zone and an incomplete overlap zone which requires a correction  

# raw signal : L1 files 
raw_signal_chm_palaiseau = xr.open_mfdataset(f"{path_data}/eprofile_1a_LpalaiseauIchm15k_v01_20220704_000011_1440.nc").rcs_0.to_dataframe()
raw_signal_chm_payerne = xr.open_mfdataset(f"{path_data}/eprofile_1a_LpayerneIchm15k_v01_20220718_000011_1440.nc").rcs_0.to_dataframe()
raw_signal_chm_granada = xr.open_mfdataset(f"{path_data}/eprofile_1a_LgranadaIchm15k_v01_20220718_000011_1440.nc").rcs_0.to_dataframe()

# overlap correction
overlap_correction_palaiseau = xr.open_dataset(f"{path_data}/Hervo_overlapmodel_palaiseau_chm15k_B_TUB140013_15m.nc").to_dataframe()
overlap_correction_granada = xr.open_dataset(f"{path_data}/Hervo_overlapmodel_granada_chm15k_A_TUB170006_15m.nc").to_dataframe()
overlap_correction_payerne = xr.open_dataset(f"{path_data}/Hervo_overlapmodel_payerne_chm15k_A_TUB140016_15m.nc").to_dataframe()

# corrected & calibrated signal 
corrected_signal_chm_palaiseau = xr.open_mfdataset(f"{path_data}/eprofile_BSC_STRATFINDER_palaiseau_chm15k-B_v01_20220704.nc").beta.to_dataframe() 
corrected_signal_chm_payerne = xr.open_mfdataset(f"{path_data}/eprofile_BSC_STRATFINDER_payerne_chm15k-A_v01_20220718.nc").beta.to_dataframe()
corrected_signal_chm_granada = xr.open_mfdataset(f"{path_data}/eprofile_BSC_STRATFINDER_granada_chm15k-A_v01_20220718.nc").beta.to_dataframe()
corrected_signal_chm_paris = xr.open_dataset(f"{path_data}/eprofile_BSC_STRATFINDER_qualair_chm15k-A_v01_20220702.nc").beta.to_dataframe()
signal_2022_07_06_palaiseau = xr.open_mfdataset(f"{path_data}/eprofile_BSC_STRATFINDER_palaiseau_chm15k-B_v01_20220706.nc").beta.to_dataframe()

# Mixed layer height retrieved by STRATfinder (MLH)
MLH_raw_signal = xr.open_mfdataset(f"{path_data}/eprofile_1a_LpalaiseauIchm15k_v01_20220704_000011_1440_L2A_noov.nc").to_dataframe()
MLH_corrected_signal = xr.open_mfdataset(f"{path_data}/eprofile_L2B_STRATFINDER_palaiseau_chm15k-B_v01_20220704.nc").to_dataframe()
MLH_2022_07_06_palaiseau = xr.open_mfdataset(f"{path_data}/eprofile_L2B_STRATFINDER_palaiseau_chm15k-B_v01_20220706.nc").to_dataframe()
MLH_CHM15k = xr.open_dataset(f"{path_data}/eprofile_1a_LqualairIchm15k_v01_20220702_000011_1440_L2A_140m.nc").to_dataframe()

In [ ]:
#5
### =========== CL61 ==============
# The CL61 provide attenuated backscatter from first gate to 15km
# No correction should be required according to Vaisala, although a near-range correction is in progress at SIRTA  

# raw signal : L1 files
L1_CL61 =  xr.open_dataset(f"{path_data}/cl61_1a_HOTE_v01_20220702_000000_1440.nc").rcs_0.to_dataframe()

# Mixed layer height derived by STRATfinder (MLH)
MLH_CL61 = xr.open_dataset(f"{path_data}/cl61_1a_HOTE_v01_20220702_000000_1440_L2A_140.nc").to_dataframe()

In [ ]:
#6
### =========== CL51 ==============
# The CL51 provide attenuated backscatter from 1st gate to ~15km, however the signal becomes very noisy from ~7km onwards and an overestimation of the signal up to ~500m requires a correction 

# raw signal : L1 files
raw_signal_cl51 = xr.open_dataset(f"{path_data}/eprofile_1a_LkarlovIcl51_v01_20190604_000011_1440.nc").rcs_0.to_dataframe()

# correction
corr_cl51 = xr.open_mfdataset(f"{path_data}/cl51_correction_karlov.nc").to_dataframe()

# corrected signal 
corrected_signal_cl51 = xr.open_mfdataset(f"{path_data}/eprofile_L1B_CABAM_karlov_cl51-A_v01_20190604.nc").RBCS.to_dataframe()
corrected_signal_cl51 = corrected_signal_cl51.reorder_levels(["time", "station", "range"]).sort_index()

# Mixed layer height derived by STRATfinder (MLH)
MLH_raw_CL51 = xr.open_mfdataset(f"{path_data}/eprofile_1a_LkarlovIcl51_v01_20190604_000011_1440_L2A_nocorr_new.nc").to_dataframe()
MLH_corrected_CL51 = xr.open_mfdataset(f"{path_data}/eprofile_1a_LkarlovIcl51_v01_20190604_000011_1440_L2A.nc").to_dataframe()


## CHM15k

The CHM15k provide attenuated backscatter from around 15m to 15km. However, for most sensors, the lowest 200m only capture noise 
(near-range blind zone) and data up to about 500m have to be corrected for the incomplete optical overlap (Hervo et al. 2016)

In [ ]:
#7
### Make the data easier to handle and lighter

# uncorrected signal
raw_signal_chm_palaiseau = from_df_to_pivot(raw_signal_chm_palaiseau, 'rcs_0', 1) 
raw_signal_chm_payerne = from_df_to_pivot(raw_signal_chm_payerne, 'rcs_0', 1) 
raw_signal_chm_granada = from_df_to_pivot(raw_signal_chm_granada, 'rcs_0', 1) 

# corrected signal 
corrected_signal_chm_palaiseau = from_df_to_pivot(corrected_signal_chm_palaiseau, 'beta', 1)
corrected_signal_chm_payerne = from_df_to_pivot(corrected_signal_chm_payerne, 'beta', 1)
corrected_signal_chm_granada = from_df_to_pivot(corrected_signal_chm_granada, 'beta', 1)

# crop data
raw_signal_chm_palaiseau = raw_signal_chm_palaiseau.loc[:3000]
raw_signal_chm_payerne = raw_signal_chm_payerne.loc[:3000]
raw_signal_chm_granada = raw_signal_chm_granada.loc[:3000]

corrected_signal_chm_palaiseau = corrected_signal_chm_palaiseau.loc[:3000]
corrected_signal_chm_payerne = corrected_signal_chm_payerne.loc[:3000]
corrected_signal_chm_granada = corrected_signal_chm_granada.loc[:3000]

# MLH resampled
MLH_raw_signal = MLH_raw_signal.droplevel("station").MLH.resample("15min").mean()
MLH_corrected_signal = MLH_corrected_signal.droplevel("station_name").MLH.resample("15min").mean()

In [ ]:
#8
### Example of the raw attenuated backscatter signal ready to be used for plot and analysis

raw_signal_chm_palaiseau

In [ ]:
#9
### Figure of the raw attenuated backscatter signal 

plt.figure(figsize=(14, 9))

img = mpimg.imread(f"{path_data}/Raw_signal_SIRTA_CHM15k.png")

plt.imshow(img)
plt.axis("off") 
plt.show()

Clear MLH develoment, in clear sky conditions

Residual layers at night

Blind zone : no signal up to ~150m

Incomplete overlap : overestimation of the signal from ~150m to ~250m

!! colorbar label should be m2*counts/s

In [ ]:
#10
### Figure of the raw attenuated backscatter signal vertical gradient 

# The gradient is taken into account by the algorithm STRATfinder to track the MLH 
# The negative gradients are potential MLH as they are associated with sharp decreases of aerosols 

df = raw_signal_chm_palaiseau.T.resample("15min").mean().T.diff().copy()

fig, ax = plt.subplots(figsize=(12, 6))

img0 = ax.pcolormesh(
    df.columns,   
    df.index,     
    df.values,    
    vmin=-3 * 10**3,
    vmax=3 * 10**3,
    cmap="viridis",
)

# Titre et labels
ax.set_title("Gradient of the attenuated backscatter signal, SIRTA 4 July 2022", fontsize=15)
ax.set_xlabel("Time")
ax.set_ylabel("Range [magl]")

# Colorbar
cbar = fig.colorbar(img0, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Normalized range corrected signal [m⁻¹ sr⁻¹]")

Areas of negative vertical gradients in attenuated backscatter indicate a decrease in aerosol content (and or humidity) and could hence mark a potential layer boundary. 
Where negative gradients are consistent in space and time, this could be linked to e.g. the height of the mixed layer or the height of the residual layer.

!!! colorbar label should be m2*counts/s

### CHM15k Overlap correction 

CHM15k has incomplete overlap up to a height of about 700 m.

Reference corrections functions are provided by manufacturer for each sensor. However, it does not capture the real variability 
of the overlap function and is hence insufficient for a high-quality overlap correction.

Recommended by ACTRIS : Temperature-dependant correction according to Hervo et al. (2016)

Requirements : at least 1 year of data to derive the temperature-dependent overlap model

Selection of clear-sky days : 1 correction per day (around 20% of days selected for 2 years in Payerne)

For one range, considering all selected days, the correction shows a clear linear dependency on the temperature (quantified by the difference between the corrected and uncorrected signal)

In Hervo et al. 2016, the lowest temperatures (e.g. in winter) lead to a correction closer to the reference correction than when high temperatures are reached

Final correction : linear fit coefficients a & b (a*Temp. + b) for each range  


In [ ]:
#11
# Overlap correction coefficients

overlap_correction_palaiseau

In [ ]:
# Example : T = 20°, a = -0.1, b -10, overlap_ref = 0.00003
# deviation = (a*T + b)/100 = -0.12
# value = overlap_ref / (1 + deviation)  

In [ ]:
#12
# Plot corrections

plot_overlap_corrections (overlap_correction_palaiseau, overlap_correction_payerne, overlap_correction_granada, 
                          raw_signal_chm_palaiseau, raw_signal_chm_payerne, raw_signal_chm_granada, 
                          corrected_signal_chm_palaiseau, corrected_signal_chm_payerne, corrected_signal_chm_granada)


Left : Overlap correction functions, 3 temperature dependant functions and the reference (dotted) 

Right : difference between raw signal VS corrected signal 

### CHM15k Calibration

To retrieve the true value of the signal and make the signals from different instruments comparable, a calibration step is required (Wiegner and Geiss, 2012).

Rayleigh method : selection of clear-sky nights, in an altitude range around 5 km, where the atmosphere is expected to contain only a very small amount of aerosols. 

In such conditions, the signal can be considered close to the theoretical molecular (Rayleigh) backscatter profile, 

which is well known (temperature and pressure references) and can therefore serve as a reference.


For each clear-sky night, a calibration constant is derived. 

The final calibration constant used for the instrument is defined as the median of all values obtained over multiple clear-sky nights. 

It should be noted, however, that this approach is still under discussion, as a seasonal cycle has been observed in the calibration values, although very likely to be instrument-related


Once the calibration constant is determined, the attenuated backscatter signal can be calculated as: Attenuated backscatter signal = Backscatter signal / calibration constant 

In [ ]:
#13
# Plot time serie of calibration constants at Fuerstenzell

plt.figure(figsize=(14, 9))

img = mpimg.imread(f"{path_data}/Calibration_constants.png")

plt.imshow(img)
plt.axis("off")  
plt.show()


The amplitude of calibration constants dependant of the laser optical module (LOM)

No impact of the firmware

Clear seasonal cycle => PROBE Cost Action VMG : Bruxmann (https://doi.org/10.5281/zenodo.11108621), Van Hove (https://doi.org/10.5281/zenodo.11074353)

In [ ]:
#14
### Plot attenuated & corrected backscatter signal 

show_MLH = 0 # 1 if you want the MLH to appear on the backscatter signal
zoom = 0

date_ = date(2022, 7, 4)
field1 = raw_signal_chm_palaiseau
field2 = corrected_signal_chm_palaiseau 
site1 = "palaiseau_raw"
site2 = "palaiseau_corrected"

data = {
    "MLH_palaiseau_raw": list(MLH_raw_signal),
    "MLH_palaiseau_corrected": list(MLH_corrected_signal),
}

df = pd.DataFrame(data, index = MLH_raw_signal.index)

vmin1 = 0
vmin2 = 0
vmax1 = 5 * 10**5
vmax2 = 1 * 10**-6

plot_MLH(date_, field1, field2, site1, site2, df, vmin1, vmin2, vmax1, vmax2, show_MLH, zoom, "magma")

## CL61


The Vaisala CL61 is a ceilometer that provides continuous vertical profiling of the atmosphere up to 15 km with a resolution of about 5m. Unlike traditional 

ceilometers, the CL61 includes depolarization measurements, which enable to determine the aerosol type such as dust, volcanic ash, or smoke. 

In theory, the CL61 is designed to deliver calibrated measurements without requiring additional corrections; 

however, in practice, some adjustments may still be necessary to ensure data consistency across different sites and conditions. 

In [ ]:
#15
### Make data easier to exploit and lighter

# CL61 Raw signal
L1_CL61 = from_df_to_pivot(L1_CL61, "rcs_0", 1)
L1_CL61 = L1_CL61.loc[:3000]

# CL61 Mixed Layer Height
MLH_CL61 = MLH_CL61.droplevel("station")
MLH_CL61 = MLH_CL61.MLH.resample("15min").mean()

# CHM15k Paris
corrected_signal_chm_paris = from_df_to_pivot(corrected_signal_chm_paris, "beta", 1)
corrected_signal_chm_paris = corrected_signal_chm_paris.loc[:3000]

# MLH Paris
MLH_CHM15k = MLH_CHM15k.droplevel("station")
MLH_CHM15k = MLH_CHM15k.MLH.resample("15min").mean()

In [ ]:
#16
### Example of data from CL61 

L1_CL61

In [ ]:
#17
# Plot CL61

show_MLH = 1
zoom = 0

date_ = date(2022, 7, 2)
field1 = corrected_signal_chm_paris
field2 = L1_CL61
site1 = "CHM15K"
site2 = "CL61"

data = {
    "MLH_CHM15K": list(MLH_CHM15k),
    "MLH_CL61": list(MLH_CL61)
}

df = pd.DataFrame(data, index = MLH_CHM15k.index)

vmin1 = 0
vmin2 = 0
vmax1 = 1 * 10**-6
vmax2 = 1 * 10**-6

plot_MLH(date_, field1, field2, site1, site2, df, vmin1, vmin2, vmax1, vmax2, show_MLH, zoom, "viridis")

About the previous figure :

Lower limit possible with CL61 : very useful for night layer

Although : depending on the blind zone height, the overlap correction quality it is possible to detected NL with CHM

Very nice agreement possible under those conditions 

In [ ]:
#18
### Same figure but zoomed in at night

vmin1 = 0
vmax1 = 1 * 10**-6
vmin2 = 0
vmax2 = 1 * 10**-6

show_MLH = 0
zoom = 1
plot_MLH(date_, field1, field2, site1, site2, df, vmin1, vmin2, vmax1, vmax2, 0, 1, "viridis")
#plot_MLH(date_, field1, field2, site1, site2, df, vmin1, vmin2, vmax1, vmax2, 1, 1, "viridis")

About the previous figure :

Careful with near-range : underestimation (or at least artifact) of the signal around 100m 

Here we set a lower limit at 140m in STRATFinder to avoid any stuck MLH near-range

## CL51
### Overlap correction

In [ ]:
#19
### Make data easier to handle and analyse

# Raw & Corrected signal 
raw_signal_cl51 = from_df_to_pivot(raw_signal_cl51, "rcs_0", 1)
corrected_signal_cl51 = from_df_to_pivot(corrected_signal_cl51, "RBCS", 1)

raw_signal_cl51 = raw_signal_cl51.loc[:3000]
corrected_signal_cl51 = corrected_signal_cl51.loc[:3000]

# MLH
MLH_raw_CL51 = MLH_raw_CL51.reset_index(level=1,drop=True).MLH
MLH_corrected_CL51 = MLH_corrected_CL51.reset_index(level=1,drop=True).MLH
MLH_raw_CL51 = MLH_raw_CL51.resample("15min").mean()
MLH_corrected_CL51 = MLH_corrected_CL51.resample("15min").mean()


In [ ]:
#20
# Example of data from a CL51

corrected_signal_cl51

In [ ]:
#21

show_MLH = 0
zoom = 0

date_ = date(2019, 6, 4)
field1 = raw_signal_cl51
field2 = corrected_signal_cl51 
site1 = "CL51_raw_signal"
site2 = "CL51_corrected_signal"

data = {
    f"MLH_{site1}": list(MLH_raw_CL51),
    f"MLH_{site2}": list(MLH_corrected_CL51),
}

df = pd.DataFrame(data, index = MLH_raw_CL51.index)

vmin1 = 0
vmin2 = 0
vmax1 = 1 * 10**2
vmax2 = 1 * 10**6

plot_MLH(date_, field1, field2, site1, site2, df, vmin1, vmin2, vmax1, vmax2, show_MLH, zoom, "magma")